In [16]:
# install dependencies
%pip install openai jinja2 ipywidgets d20 python-dotenv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [17]:
# load dotenv file
import os

from dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY") is None:
    raise Exception("missing OPENAI API KEY...")

In [18]:
# load config file
import tomllib

config_name = "config_cn.toml"
config:dict[str, any] = None
with open(config_name, "rb") as f:
    config = tomllib.load(f)

print(f"{config_name} loaded: v{config["version"]}")

config_cn.toml loaded: v0.0.2


In [19]:
# load config
from jinja2 import BaseLoader, Environment

meta = config.get("meta", None)
if meta.get("memories") is None:
    meta["memories"] = []

current_round = 0
messages:dict[str, str] = []

def next_round():
    global current_round, messages
    messages = [] # reset messages
    current_round += 1
    for msg in config["messages"]:
        role = msg.get("role")
        raw_content = msg.get("content")
        # print(f"<< {role} >>")
        
        template = Environment(loader=BaseLoader()).from_string(raw_content)
        content = template.render(meta)
        # print(content)

        messages.append({"role": role, "content": content})
        # print("------")
    
    for msg in config["rounds"][current_round - 1]["messages"]:
        role = msg.get("role")
        raw_content = msg.get("content")
        # print(f"<< {role} >>")

        template = Environment(loader=BaseLoader()).from_string(raw_content)
        content = template.render(meta)
        # print(content)
       
        messages.append({"role": role, "content": content})
        # print("------")

next_round()

In [20]:
# start interactive chat
import re
import functools

import ipywidgets as widgets
from IPython.display import display

from openai import OpenAI
import d20

def count_user_actions():
    count = 0
    for msg in messages:
        if msg["role"] == "user":
            count += 1
    return count

def is_final_round():
    return  ", FINAL ROUND" if  current_round == len(config["rounds"]) else ""

def on_custom_input(b, input:widgets.Text):
    role = "user"
    content = f"{input.value} (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_action(b, index:str):
    role = "user"
    content = f"I have chosen action {index}. (TURN: {count_user_actions()+1}{is_final_round()})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def on_skill(b, skill:str, difficulty:str):
    role = "user"
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. (TURN: {count_user_actions()+1}, {is_final_round()})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_next_round(b):
    next_round()
    print("ROUND:", current_round)
    print(f"{messages[-1]["role"].upper()}:")
    print(messages[-1]["content"])
    parse_message(messages[-1]["content"])

# parse message into key-value pairs
pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
action_pattern = re.compile(r"(\d+). *(.+)")
def parse_message(content):
    parsed: dict = {}
    current_key = None
    lines = content.splitlines()
    
    # parse the content line by line
    for line in lines:
        res = pattern.search(line)
        if res is not None:
            current_key = res.group(1)
            if res.group(2) is not None:
                parsed[current_key] = res.group(2)
            else:
                parsed[current_key] = ""
        elif current_key is not None:
            parsed[current_key] += "\n" + line
    
    keys = parsed.keys()
    # possible actions
    vbox = widgets.HBox()
    if "possible actions" in keys:
        ma = action_pattern.finditer(parsed["possible actions"])
        for m in ma:
            action_index=m.group(1)
            action_button = widgets.Button(description=action_index)
            action_button.on_click(functools.partial(on_action, index=action_index))
            vbox.children += (action_button,)
        custom_action_button = widgets.Button(description="CUSTOM")
        custom_action_button.on_click(functools.partial(on_custom_action))
        
        vbox.children += (custom_action_button,)
        display(vbox)
    
    # skill check
    if "skill" in keys and "difficulty" in keys:
        skill = parsed.get("skill")
        difficulty = parsed.get("difficulty")
        skill_button = widgets.Button(description=f"SKILL CHECK - {skill.upper()}", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        display(skill_button)
    
    if "round summary" in keys:
        summary = parsed["round summary"]
        next_round_button = widgets.Button(description="NEXT ROUND")
        meta["memories"].append(summary)
        
        next_round_button.on_click(on_next_round)
        display(next_round_button)
    
    if "ending summary" in keys:
        print("\n\nGAME OVER\n\n(try to re-run the cell to start a new game)")

def chat():
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )
    print("\nASSISTANT:")
    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            print(delta, end="")
            generated += delta
    messages.append({"role": "assistant", "content": generated})
    parse_message(messages[-1]["content"])

print("ROUND:", current_round)
print(f"{messages[-1]["role"].upper()}:")
print(messages[-1]["content"])
# parse the last message
parse_message(messages[-1]["content"])

ROUND: 1
ASSISTANT:
你站在闹鬼的豪宅前。
这座豪宅古老而破旧，墙上爬满了爬山虎，窗户被钉上了木板。
前门微微敞开，邀请你进入。

你要做什么？

possible actions:
1. 进入豪宅。
2. 拍摄豪宅外部的照片。
3. 与管家交谈。(可能需要一个说服技能检定)
